In [1]:
from tensorflow.keras import layers
import tensorflow as tf

from src.models.lstm import Seq2SeqAttentionLSTM, Seq2SeqLSTM
from src.data.scan import (
    MAX_SEQUENCE_LENGTH,
    IN_VOCAB_FILE,
    POS_VOCAB_SIZE,
    OUT_VOCAB_SIZE,
    IN_VOCAB_SIZE,
    POS_VOCAB_FILE,
    OUT_VOCAB_FILE,
    load_dataset,
    get_add_pos_tag,
    DS_POS_FILE,
    add_sos_eos,
    get_vectorizer,
    get_final_map_function
)
from src.utils.constants import ACTION_INPUT_NAME, ACTION_OUTPUT_NAME, COMMAND_INPUT_NAME, POS_OUTPUT_NAME, POS_INPUT_NAME

In [2]:
from tensorflow.keras import backend as K
def filter(x, *_):
    return K.any(K.equal(x, 'jump'))

def get_dataset(experiment: str):
    train, test = load_dataset(experiment, split=["train", "test"])

    # Split dataset into tuples
    train = train.map(lambda x: (x["commands"], x["actions"]), num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(lambda x: (x["commands"], x["actions"]), num_parallel_calls=tf.data.AUTOTUNE)

    train = train.filter(filter)
    # test = test.filter(filter)
    
    add_pos_tag = get_add_pos_tag(DS_POS_FILE)

    # Add pos tag to dataset
    train = train.map(add_pos_tag, num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(add_pos_tag, num_parallel_calls=tf.data.AUTOTUNE)

    # Add <sos> and <eos> to dataset
    train = train.map(add_sos_eos, num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(add_sos_eos, num_parallel_calls=tf.data.AUTOTUNE)

    vectorize_text, in_vectorizer, pos_vectorizer, out_vectorizer = get_vectorizer()

    # Vectorize dataset
    train = train.map(
        lambda x, p, y: tf.py_function(vectorize_text, inp=(x, p, y), Tout=(tf.int64, tf.int64, tf.int64)),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    test = test.map(
        lambda x, p, y: tf.py_function(vectorize_text, inp=(x, p, y), Tout=(tf.int64, tf.int64, tf.int64)),
        num_parallel_calls=tf.data.AUTOTUNE,
    )


    return train, test, (in_vectorizer, pos_vectorizer, out_vectorizer)

In [3]:
in_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=IN_VOCAB_SIZE, standardize=None
)
pos_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=POS_VOCAB_SIZE, standardize=None
)
out_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=OUT_VOCAB_SIZE, standardize=None
)


in_vectorizer.set_vocabulary(IN_VOCAB_FILE)
pos_vectorizer.set_vocabulary(POS_VOCAB_FILE)
out_vectorizer.set_vocabulary(OUT_VOCAB_FILE)

in_voc = in_vectorizer.get_vocabulary()
out_voc = out_vectorizer.get_vocabulary()
pos_voc = pos_vectorizer.get_vocabulary()

In [4]:
model_in = "<sos> jump and run <eos>"
model_pos = "NOUN CONJ VERB"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
model_pos_vec = pos_vectorizer(tf.convert_to_tensor([model_pos, model_pos]))

In [5]:
hidden_size = 100
hidden_layers = 1
include_pos_tag = "aux"
use_attention = True

Model = Seq2SeqAttentionLSTM if use_attention else Seq2SeqLSTM

model = Model(
    hidden_size=hidden_size,
    hidden_layers=hidden_layers,
    include_pos_tag=include_pos_tag,
    teacher_forcing=0,
)

pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)
checkpoint_path = f"snap/simple-h_size({hidden_size})-h_layers({hidden_layers})-dropout(0.1){f'-pos({include_pos_tag})' if include_pos_tag else ''}{'-attention' if use_attention else ''}/best_action_accuracy"
model.load_weights(checkpoint_path)

In [22]:
model_in = "<sos> jump and run <eos>"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))

In [14]:
train, test, _ = get_dataset("simple")
final_map_fn = get_final_map_function(
    include_pos_tag="aux",
    teacher_forcing=False,
)
train = train.map(final_map_fn)
test = test.map(final_map_fn)

In vocabulary: {0: '', 1: '[UNK]', 2: '<sos>', 3: '<eos>', 4: 'run', 5: 'opposite', 6: 'right', 7: 'after', 8: 'turn', 9: 'left', 10: 'twice', 11: 'around', 12: 'thrice', 13: 'walk', 14: 'jump', 15: 'and', 16: 'look'}
Pos vocabulary: {0: '', 1: '[UNK]', 2: '<sos>', 3: '<eos>', 4: 'ADV', 5: 'CNJ', 6: 'DIR', 7: 'MOD', 8: 'VRB'}
Out vocabulary: {0: '', 1: '[UNK]', 2: '<sos>', 3: '<eos>', 4: 'I_TURN_LEFT', 5: 'I_TURN_RIGHT', 6: 'I_RUN', 7: 'I_WALK', 8: 'I_JUMP', 9: 'I_LOOK'}


In [23]:
def get_pred(val, model):
    x, y = val
    print(x.keys(), x[COMMAND_INPUT_NAME].shape)
    print(y.keys())
    res = model(x, training=False)
    
    in_seq1 = x[COMMAND_INPUT_NAME][0]
    in_seq1 = " ".join([in_voc[i] for i in in_seq1]).strip()

    actions1 = tf.argmax(res[ACTION_OUTPUT_NAME][0], axis=-1)
    actions1 = " ".join([out_voc[i] for i in actions1]).strip()
    actions1 = actions1.split('<eos>')[0]

    in_seq2 = x[COMMAND_INPUT_NAME][1]
    in_seq2 = " ".join([in_voc[i] for i in in_seq2]).strip()

    actions2 = tf.argmax(res[ACTION_OUTPUT_NAME][1], axis=-1)
    actions2 = " ".join([out_voc[i] for i in actions2]).strip()
    actions2 = actions2.split('<eos>')[0]

    return in_seq1, actions1, in_seq2, actions2

In [45]:
def stuff(ds, model):
    z = ds.take(10).batch(2)
    for vals in z:
        print(vals)
        i, o, i2, o2 = get_pred(vals, model)
        print(i)
        print(o)
        print(i2)
        print(o2)
        yield
    print('wtf')

In [46]:
s = stuff(train, model)

In [47]:
next(s)

TypeError: 'TakeDataset' object is not an iterator

In [46]:
i, o, i2, o2 = get_pred(ds, model)
print(i)
print(o)
print(i2)
print(o2)


dict_keys(['input_embedding'])
dict_keys(['action_output', 'pos_output'])
[ 2 14  3  0  0  0  0  0  0  0  0  0]
<sos> jump <eos>
<sos> <sos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>


In [16]:
model_in = "<sos> run around left after walk left <eos>"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)

actions = tf.argmax(pre_res[ACTION_OUTPUT_NAME][0], axis=-1)
# pos = tf.argmax(pre_res[POS_OUTPUT_NAME][0], axis=-1)

actions = " ".join([out_voc[i] for i in actions]).strip()
# pos = " ".join([pos_voc[i] for i in pos]).strip()

print(actions)
# print(pos)

<sos> I_TURN_LEFT I_WALK I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
